# Predicting Students' dropout and academic success

In [91]:
# import packages and create dataframe
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

print('Packages imported successfully.')

Packages imported successfully.


In [92]:
# The code was removed by Watson Studio for sharing.

In [93]:
# read csv and create dataframe
df_data_1 = pd.read_csv(body, delimiter=';')
df_data_1.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [94]:
df = df_data_1.copy(deep=True) # make a deep copy to work with (keep original unchaged)

In [95]:
df.info() # get some general information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance	                     4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   int64  
 8   Mother's qualification                          4424 non-null   int64  
 9   Father's qualification                   

In [96]:
df.shape # dataframe's shape

(4424, 37)

In [97]:
# some columns are obviously nonrelevant so we drop them
df.drop(columns=['Course', 'Nacionality'], inplace=True)
df.shape # new shape

(4424, 35)

In [98]:
df.isnull().sum() # get the sum of nulls

Marital status                                    0
Application mode                                  0
Application order                                 0
Daytime/evening attendance\t                      0
Previous qualification                            0
Previous qualification (grade)                    0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Admission grade                                   0
Displaced                                         0
Educational special needs                         0
Debtor                                            0
Tuition fees up to date                           0
Gender                                            0
Scholarship holder                                0
Age at enrollment                                 0
International                                     0
Curricular u

In [99]:
df.Target.value_counts() # get the uniques in Target (label) col

Graduate    2209
Dropout     1421
Enrolled     794
Name: Target, dtype: int64

In [100]:
# create dataframe for enrolled students
df_enrolled = df[df['Target']=='Enrolled']
df_enrolled.head()

,Marital status,Application mode,Application order,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
16,1,18,1,1,1,137.0,19,38,5,8,...,0,6,14,2,11.000,0,10.8,1.4,1.74,Enrolled
19,1,1,1,1,1,140.0,19,19,7,7,...,0,7,8,6,13.500,0,16.2,0.3,-0.92,Enrolled
21,1,18,4,1,1,127.0,1,38,4,7,...,0,8,9,8,11.425,0,12.7,3.7,-1.70,Enrolled
25,1,1,1,1,1,151.0,19,38,9,9,...,0,6,12,4,11.000,0,7.6,2.6,0.32,Enrolled
27,1,1,1,1,1,138.0,19,19,3,3,...,0,5,7,4,13.000,0,9.4,-0.8,-3.12,Enrolled


In [101]:
df_enrolled.shape # dataframe's shape

(794, 35)

In [102]:
# remove enrolled students from dataframe (to be predicted later)
df = df.drop(df.index[df['Target'] == 'Enrolled'])
df.Target.value_counts()

Graduate    2209
Dropout     1421
Name: Target, dtype: int64

In [103]:
# imbalanced dataset!

In [104]:
2209-1421 # difference

788

In [105]:
(2209-1421)/(2209+1421) # 22% more graduates than dropouts!

0.21707988980716253

In [106]:
from sklearn.utils import resample
df_majority = df[df['Target'] == 'Graduate']
df_minority = df[df['Target'] == 'Dropout']
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=2209,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.Target.value_counts()

Graduate    2209
Dropout     2209
Name: Target, dtype: int64

In [107]:
# the above will construct training and evaluation data (trainset, testset)

In [108]:
# convert categorical Target into numeric values
df_upsampled['Target'].replace(['Graduate', 'Dropout'], [1,0], inplace=True)
# check class distribution
df_upsampled.Target.value_counts()

1    2209
0    2209
Name: Target, dtype: int64

In [109]:
df_upsampled.describe() # stats

,Marital status,Application mode,Application order,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
count,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,...,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000,4418.000000
mean,1.207107,19.503622,1.713898,0.880715,4.752603,132.619036,20.360344,22.766184,10.178814,10.197148,...,0.582617,6.234722,7.700091,4.052286,9.302033,0.160027,11.613762,1.250792,0.000962,0.500000
std,0.646738,17.541970,1.310894,0.324160,10.127538,13.370564,15.570622,15.219676,22.930577,21.915532,...,2.026111,2.267068,4.162959,3.237869,5.819446,0.794175,2.696336,1.380873,2.236289,0.500057
min,1.000000,1.000000,0.000000,0.000000,1.000000,95.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,124.000000,3.000000,3.000000,4.000000,4.000000,...,0.000000,5.000000,6.000000,0.000000,0.000000,0.000000,9.400000,0.300000,-1.700000,0.000000
50%,1.000000,17.000000,1.000000,1.000000,1.000000,133.100000,19.000000,19.000000,5.000000,7.000000,...,0.000000,6.000000,8.000000,5.000000,12.000000,0.000000,11.100000,1.400000,0.320000,0.500000
75%,1.000000,39.000000,2.000000,1.000000,1.000000,140.000000,37.000000,37.000000,9.000000,9.000000,...,0.000000,7.000000,10.000000,6.000000,13.300000,0.000000,13.900000,2.600000,1.790000,1.000000
max,6.000000,57.000000,6.000000,1.000000,43.000000,188.000000,43.000000,44.000000,194.000000,195.000000,...,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,16.200000,3.700000,3.510000,1.000000


In [110]:
corr = df_upsampled.corr(method='pearson')['Target'] # correlation matrix
corr

Marital status                                   -0.111657
Application mode                                 -0.255731
Application order                                 0.104647
Daytime/evening attendance\t                      0.087292
Previous qualification                           -0.071236
Previous qualification (grade)                    0.109482
Mother's qualification                           -0.067371
Father's qualification                           -0.017506
Mother's occupation                              -0.001125
Father's occupation                               0.008067
Admission grade                                   0.143716
Displaced                                         0.129782
Educational special needs                         0.034174
Debtor                                           -0.271461
Tuition fees up to date                           0.413393
Gender                                           -0.264461
Scholarship holder                                0.3380

In [111]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

In [112]:
mms = MinMaxScaler() # normalize dataframe
for col in df_upsampled.columns:
    mms.fit(np.array(df_upsampled[col]).reshape(-1,1))
    df_upsampled[col]=mms.transform(np.array(df_upsampled[col]).reshape(-1,1))

In [113]:
df_upsampled.head() # normalized

,Marital status,Application mode,Application order,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
1,0.0,0.250000,0.166667,1.0,0.000000,0.698925,0.000000,0.046512,0.015464,0.015385,...,0.0,0.260870,0.181818,0.30,0.735897,0.000000,0.732558,0.111111,0.640687,1.0
3,0.0,0.285714,0.333333,1.0,0.000000,0.290323,0.880952,0.837209,0.025773,0.015385,...,0.0,0.260870,0.303030,0.25,0.667692,0.000000,0.209302,0.000000,0.124174,1.0
4,0.2,0.678571,0.166667,0.0,0.000000,0.053763,0.857143,0.860465,0.046392,0.046154,...,0.0,0.260870,0.181818,0.30,0.700000,0.000000,0.732558,0.111111,0.640687,1.0
5,0.2,0.678571,0.166667,0.0,0.428571,0.409677,0.857143,0.837209,0.046392,0.035897,...,0.0,0.217391,0.515152,0.25,0.619231,0.416667,1.000000,0.244444,0.414795,1.0
6,0.0,0.000000,0.166667,1.0,0.000000,0.505376,0.428571,0.860465,0.036082,0.051282,...,0.0,0.347826,0.242424,0.40,0.772423,0.000000,0.918605,0.800000,0.000000,1.0


In [114]:
# features with a correlation of >= .1
attr = [_ for _ in corr.T.index if abs(corr.T.loc[_]) >= .1]

features = attr[:-1]
target = attr[-1]

print(features)
print(target)

['Marital status', 'Application mode', 'Application order', 'Previous qualification (grade)', 'Admission grade', 'Displaced', 'Debtor', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'Age at enrollment', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)']
Target


In [115]:
X = df_upsampled[features]
y = df_upsampled[target]
print(X[:5])
print(90*'*')
print(y[:5])

   Marital status  Application mode  Application order  \
1             0.0          0.250000           0.166667   
3             0.0          0.285714           0.333333   
4             0.2          0.678571           0.166667   
5             0.2          0.678571           0.166667   
6             0.0          0.000000           0.166667   

   Previous qualification (grade)  Admission grade  Displaced  Debtor  \
1                        0.698925         0.500000        1.0     0.0   
3                        0.290323         0.258947        1.0     0.0   
4                        0.053763         0.489474        0.0     0.0   
5                        0.409677         0.208421        0.0     1.0   
6                        0.505376         0.351579        1.0     0.0   

   Tuition fees up to date  Gender  Scholarship holder  Age at enrollment  \
1                      0.0     1.0                 0.0           0.037736   
3                      1.0     0.0                 0.0    

In [116]:
from sklearn.model_selection import train_test_split

In [117]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [118]:
X_train.head()

,Marital status,Application mode,Application order,Previous qualification (grade),Admission grade,Displaced,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,Curricular units 1st sem (enrolled),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations)
1749,0.0,0.750000,0.166667,0.139785,0.107368,1.0,0.0,1.0,1.0,0.0,0.150943,0.653846,0.500000,0.611309,0.652174,0.454545,0.50,0.667692,0.0
3369,0.0,0.285714,0.166667,0.591398,0.502105,0.0,1.0,1.0,0.0,0.0,0.018868,0.230769,0.153846,0.582781,0.260870,0.363636,0.15,0.610256,0.0
4013,0.0,0.892857,0.166667,0.365591,0.450526,1.0,0.0,1.0,0.0,1.0,0.056604,0.269231,0.269231,0.854645,0.347826,0.272727,0.40,0.835812,0.0
2365,0.0,0.303571,0.166667,0.322581,0.278947,1.0,0.0,1.0,1.0,0.0,0.037736,0.230769,0.230769,0.732892,0.260870,0.181818,0.30,0.780769,0.0
4007,0.0,0.678571,0.166667,0.409677,0.157895,0.0,0.0,1.0,1.0,0.0,0.264151,0.230769,0.115385,0.762914,0.260870,0.606061,0.00,0.000000,0.0


In [119]:
y_train.head()

1749    0.0
3369    0.0
4013    1.0
2365    1.0
4007    0.0
Name: Target, dtype: float64

In [120]:
X_test.head()

,Marital status,Application mode,Application order,Previous qualification (grade),Admission grade,Displaced,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,Curricular units 1st sem (enrolled),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations)
2122,0.0,0.000000,0.166667,0.290323,0.269474,1.0,0.0,1.0,0.0,1.0,0.037736,0.192308,0.192308,0.678146,0.217391,0.242424,0.25,0.624615,0.0
4194,0.0,0.000000,0.833333,0.290323,0.236842,1.0,1.0,0.0,0.0,0.0,0.094340,0.192308,0.076923,0.609272,0.217391,0.363636,0.00,0.000000,0.0
2446,0.0,0.107143,0.166667,0.591398,0.578947,1.0,0.0,1.0,1.0,0.0,0.509434,0.269231,0.269231,0.715232,0.217391,0.181818,0.25,0.872308,0.0
3572,0.0,0.750000,0.166667,0.409677,0.263158,0.0,0.0,1.0,1.0,0.0,0.207547,0.500000,0.269231,0.544939,0.478261,0.393939,0.25,0.570769,0.0
3710,0.0,0.285714,0.666667,0.451613,0.490526,0.0,0.0,1.0,1.0,0.0,0.075472,0.192308,0.153846,0.649007,0.217391,0.212121,0.25,0.646154,0.0


In [121]:
y_test.head()

2122    1.0
4194    0.0
2446    0.0
3572    0.0
3710    1.0
Name: Target, dtype: float64

In [122]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2960, 19) (2960,)
(1458, 19) (1458,)


In [123]:
upper_tri = X_train.corr().where(np.triu(np.ones(X_train.corr().shape),k=1).astype(np.bool))
upper_tri # create correlation matrix on training set and select the upper triangle

,Marital status,Application mode,Application order,Previous qualification (grade),Admission grade,Displaced,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,Curricular units 1st sem (enrolled),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations)
Marital status,NaN,0.252834,-0.132580,-0.007927,-0.026904,-0.224184,0.036888,-0.125633,-0.005521,-0.074994,0.488952,0.063068,-0.048643,-0.071548,0.048586,0.013030,-0.066171,-0.099766,0.032423
Application mode,NaN,NaN,-0.299133,-0.107306,-0.091250,-0.300839,0.112954,-0.168373,0.211195,-0.173908,0.533493,0.162228,-0.057174,-0.123653,0.132276,0.164727,-0.109886,-0.146635,0.062927
Application order,NaN,NaN,NaN,-0.054805,-0.080772,0.338329,-0.073517,0.064328,-0.127427,0.089486,-0.291990,-0.009398,0.047627,0.065115,0.033548,-0.054665,0.087950,0.077709,-0.021404
Previous qualification (grade),NaN,NaN,NaN,NaN,0.581830,-0.008745,-0.062140,0.083960,-0.057679,0.071679,-0.124848,-0.084488,0.004971,0.038217,-0.087114,-0.100893,0.010718,0.023921,-0.034546
Admission grade,NaN,NaN,NaN,NaN,NaN,0.024768,-0.034410,0.085896,-0.000317,0.028619,-0.065885,-0.067902,0.070012,0.077502,-0.073753,-0.096044,0.086259,0.080091,-0.040898
Displaced,NaN,NaN,NaN,NaN,NaN,NaN,-0.126929,0.142628,-0.136132,0.077730,-0.363772,-0.064181,0.074724,0.074526,-0.045541,-0.027457,0.096457,0.102138,-0.030927
Debtor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.440113,0.093599,-0.063838,0.116370,-0.059548,-0.145214,-0.107399,-0.083351,0.020071,-0.201449,-0.178802,0.086068
Tuition fees up to date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.158962,0.167918,-0.214992,0.075319,0.283062,0.265810,0.104188,0.047857,0.345320,0.325428,-0.080550
Gender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.184528,0.189956,-0.078808,-0.204070,-0.201024,-0.107014,-0.018928,-0.239937,-0.218328,0.067795
Scholarship holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.218729,-0.024948,0.173876,0.205036,0.004631,-0.010196,0.222773,0.214042,-0.067843


In [124]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
to_drop # check if some features have a strong correlation (>.95) to drop - no correlation (good)

[]

In [125]:
# We are now good to go for modelling

In [126]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier() # create a model

In [127]:
model.fit(X_train, y_train) # train model

GradientBoostingClassifier()

In [128]:
from sklearn.metrics import classification_report
print("Model training results:")

# Report an error rate on the training set
report_train = classification_report(y_train, model.predict(X_train))
print(report_train)

Model training results:
              precision    recall  f1-score   support

         0.0       0.96      0.91      0.93      1497
         1.0       0.91      0.96      0.94      1463

    accuracy                           0.94      2960
   macro avg       0.94      0.94      0.94      2960
weighted avg       0.94      0.94      0.94      2960



In [129]:
# Report an error rate on the test set
print("Model test results:")
# Report an error rate on the training set
report_test = classification_report(y_test, model.predict(X_test))
print(report_test)

Model test results:
              precision    recall  f1-score   support

         0.0       0.92      0.88      0.90       712
         1.0       0.89      0.93      0.91       746

    accuracy                           0.91      1458
   macro avg       0.91      0.91      0.91      1458
weighted avg       0.91      0.91      0.91      1458



In [130]:
# The reports both for training and test set suggest good results
# so we save the model as file
import joblib
joblib.dump(model, 'students_dropout_prediction.pkl')

['students_dropout_prediction.pkl']

In [131]:
# before making predictions we need to normalize the enrolled dataframe as well:

In [132]:
minmax = MinMaxScaler() # normalize dataframe
for col in df_enrolled.columns[:-1]:
    minmax.fit(np.array(df_enrolled[col]).reshape(-1,1))
    df_enrolled[col]=minmax.transform(np.array(df_enrolled[col]).reshape(-1,1))

In [133]:
df_enrolled.head()

,Marital status,Application mode,Application order,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
16,0.0,0.326923,0.000,1.0,0.0,0.436170,0.428571,0.925,0.025773,0.041451,...,0.0,0.352941,0.500000,0.166667,0.625000,0.0,0.372093,0.488889,0.766182,Enrolled
19,0.0,0.000000,0.000,1.0,0.0,0.468085,0.428571,0.450,0.036082,0.036269,...,0.0,0.411765,0.285714,0.500000,0.767045,0.0,1.000000,0.244444,0.414795,Enrolled
21,0.0,0.326923,0.375,1.0,0.0,0.329787,0.000000,0.925,0.020619,0.036269,...,0.0,0.470588,0.321429,0.666667,0.649148,0.0,0.593023,1.000000,0.311757,Enrolled
25,0.0,0.000000,0.000,1.0,0.0,0.585106,0.428571,0.925,0.046392,0.046632,...,0.0,0.352941,0.428571,0.333333,0.625000,0.0,0.000000,0.755556,0.578600,Enrolled
27,0.0,0.000000,0.000,1.0,0.0,0.446809,0.428571,0.450,0.015464,0.015544,...,0.0,0.294118,0.250000,0.333333,0.738636,0.0,0.209302,0.000000,0.124174,Enrolled


In [143]:
# using model to make predictions on enrolled dataset
predict_enrolled = model.predict(df_enrolled[features])
predict_enrolled[:10] # show the first ten results

array([0., 1., 1., 1., 1., 1., 1., 0., 0., 1.])

In [135]:
results = pd.Series(predict_enrolled).value_counts()
results

1.0    478
0.0    316
dtype: int64

In [138]:
print(f"according to our model's prediction {results[1.0]} students will graduate and {results[0.0]} will dropout.")

according to our model's prediction 478 students will graduate and 316 will dropout.


In [139]:
# we may also pridict the probabilities
model.predict_proba(df_enrolled[features])

array([[0.9621192 , 0.0378808 ],
       [0.06855114, 0.93144886],
       [0.0395283 , 0.9604717 ],
       ...,
       [0.92913815, 0.07086185],
       [0.03739647, 0.96260353],
       [0.97756654, 0.02243346]])

In [144]:
# the first number in each row shows the probabilty of a succesful graduation
# the second number indicates the probability of a droput

In [145]:
# we may add the predictions to the enrolled dataframe as a new column
df_enrolled['Prediction'] = predict_enrolled
df_enrolled.head(10)

,Marital status,Application mode,Application order,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,...,Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,Prediction
16,0.0,0.326923,0.000,1.0,0.000000,0.436170,0.428571,0.925,0.025773,0.041451,...,0.352941,0.500000,0.166667,0.625000,0.000,0.372093,0.488889,0.766182,Enrolled,0.0
19,0.0,0.000000,0.000,1.0,0.000000,0.468085,0.428571,0.450,0.036082,0.036269,...,0.411765,0.285714,0.500000,0.767045,0.000,1.000000,0.244444,0.414795,Enrolled,1.0
21,0.0,0.326923,0.375,1.0,0.000000,0.329787,0.000000,0.925,0.020619,0.036269,...,0.470588,0.321429,0.666667,0.649148,0.000,0.593023,1.000000,0.311757,Enrolled,1.0
25,0.0,0.000000,0.000,1.0,0.000000,0.585106,0.428571,0.925,0.046392,0.046632,...,0.352941,0.428571,0.333333,0.625000,0.000,0.000000,0.755556,0.578600,Enrolled,1.0
27,0.0,0.000000,0.000,1.0,0.000000,0.446809,0.428571,0.450,0.015464,0.015544,...,0.294118,0.250000,0.333333,0.738636,0.000,0.209302,0.000000,0.124174,Enrolled,1.0
29,0.0,0.307692,0.125,1.0,0.000000,0.329787,0.047619,0.050,0.005155,0.051813,...,0.411765,0.250000,0.500000,0.779356,0.000,1.000000,0.244444,0.414795,Enrolled,1.0
30,0.0,0.826923,0.000,1.0,0.926829,0.574468,0.071429,0.100,0.020619,0.010363,...,0.352941,0.607143,0.416667,0.600649,0.000,1.000000,0.244444,0.414795,Enrolled,1.0
32,0.0,0.000000,0.000,1.0,0.000000,0.500000,0.000000,0.000,0.020619,0.031088,...,0.294118,0.285714,0.166667,0.767045,0.000,0.151163,0.488889,1.000000,Enrolled,0.0
45,0.0,0.307692,0.000,0.0,0.000000,0.617021,0.880952,0.925,0.020619,0.025907,...,0.294118,0.392857,0.250000,0.814394,0.125,0.000000,0.755556,0.578600,Enrolled,0.0
51,0.0,0.000000,0.000,1.0,0.000000,0.457447,0.857143,0.925,0.030928,0.031088,...,0.470588,0.357143,0.500000,0.672348,0.000,0.209302,0.000000,0.124174,Enrolled,1.0
